In [2]:
import csv
import time

import matplotlib.pyplot as plt
import torch
from scripts import load_dataset
from scripts.encoder_model_wrapper import BertTextClassifier
from torch import nn, optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model = BertTextClassifier(loss_fn=loss_fn, lr=2e-7, use_gradient_clip=True)

In [ ]:
optimizer = optim.AdamW(model.parameters(), model.lr)

In [ ]:
def tokenize_function(examples):
    return model.tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
train_ds, _ = load_dataset.load_dataset(
    "../German_newspaper_articles/10kGNAD/train.csv",
    "../German_newspaper_articles/10kGNAD/test.csv",
)

In [ ]:
train_ds = train_ds.map(tokenize_function)

In [ ]:
train_ds = train_ds.map(load_dataset.add_label_id)

In [ ]:
train_ds.set_format("torch", device=device)
model.to(device)

In [ ]:
def eval(val_data):
    model.eval()
    correct = 0
    all = 0
    for data in val_data:
        label_id = torch.unsqueeze(data["label_id"], 0)
        output = model(
            torch.unsqueeze(data["input_ids"], 0),
            torch.unsqueeze(data["attention_mask"], 0),
        )
        output = torch.argmax(output)
        if label_id == output:
            correct += 1
        all += 1
    print(f"Eval accuracy: {(correct/all)*100:.2f}%")
    return (correct / all) * 100

In [ ]:
batch_size = 8
batch_index = 0
running_loss = 0
epochs = 10
train_eval = train_ds.train_test_split(test_size=0.2, shuffle=True)
loss_ls = []
accuracy_ls = []
start_time = time.perf_counter()
for i in range(epochs):
    print(f"Epoch: {i}")
    train_eval = train_eval.shuffle()
    for j in range(int(len(train_eval["train"]) / batch_size)):
        model.train()
        batch = train_eval["train"][batch_index : batch_index + batch_size]
        batch_index += batch_size
        optimizer.zero_grad()

        output = model(batch["input_ids"], batch["attention_mask"])
        loss = model.loss_fn(output, batch["label_id"])
        loss.backward()
        if model.use_gradient_clip:
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        running_loss += loss.item()
        if j % 50 == 49:
            print(f"loss: {running_loss/50}")
            loss_ls.append(running_loss / 50)
            running_loss = 0
        if j % 200 == 199:
            accuracy_ls.append(eval(train_eval["test"]))

    batch_index = 0

end_time = time.perf_counter()
print(f"Training took {(end_time - start_time)/60:0.4f} minutes")

In [ ]:
metrics = {"loss": loss_ls, "accuracy": accuracy_ls}
path_to_save_metrics = "../../../results/bert_results/metrics"
with open(
    path_to_save_metrics + "/training_progress.csv", "w", encoding="utf-8"
) as file:
    file_writer = csv.DictWriter(file, fieldnames=metrics.keys())
    file_writer.writeheader()
    file_writer.writerow(metrics)

In [ ]:
loss_steps = [(i + 1) * 50 for i in range(len(loss_ls))]
accuracy_steps = [(i + 1) * 200 for i in range(len(accuracy_ls))]

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
plt.gca().ticklabel_format(axis="both", style="plain", useOffset=False)
ax1.plot(loss_steps, loss_ls, "r-")
ax2.plot(accuracy_steps, accuracy_ls, "g-")
ax1.set_xlabel("Steps")
ax1.set_ylabel("Loss", color="r")
ax2.set_ylabel("Accuracy", color="g")

plt.title("GBert large")

plt.savefig(path_to_save_metrics + "/training_progress.png")
plt.show()

In [ ]:
path_to_model = "../../../results/bert_results/model.nosync/model.pt"
torch.save(model, path_to_model)